In [1]:
import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
from glove import Glove
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense

import geopandas as gpd
import os

import json
import h5py

labelEncoder = LabelEncoder()
one_enc = OneHotEncoder()
lemma = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/ranee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ranee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ranee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


## Manual Classification

In [2]:
#Dir = '/mnt/d/Dropbox/Ranee_Joshi_PhD_Local/04_PythonCodes/dh2loop_old/shp_NSW'
#DF=litho_Dataframe(Dir)
#DF.to_csv('export.csv')
DF = pd.read_csv('/mnt/d/Dropbox/Ranee_Joshi_PhD_Local/04_PythonCodes/dh2loop/notebooks/Upscaled_Litho_Test2.csv')
DF['FromDepth'] = pd.to_numeric(DF.FromDepth)
DF['ToDepth'] = pd.to_numeric(DF.ToDepth)
DF['TopElev'] = pd.to_numeric(DF.TopElev)
DF['BottomElev'] = pd.to_numeric(DF.BottomElev)
DF['x'] = pd.to_numeric(DF.x)
DF['y'] = pd.to_numeric(DF.y)
print('number of original litho classes:', len(DF.MajorLithCode.unique()))
print('number of litho classes :',
          len(DF['reclass'].unique()))
print('unclassified descriptions:',
          len(DF[DF['reclass'].isnull()]))

number of original litho classes: 340
number of litho classes : 78
unclassified descriptions: 0


In [3]:
def save_file(DF, name):
    '''Function to save manually reclassified dataframe
    Inputs:
        -DF: reclassified pandas dataframe
        -name: name (string) to save dataframe file
    '''
    DF.to_pickle('{}.pkl'.format(name))

In [4]:
save_file(DF, 'manualTest_ygsb')

## MLP Classification

In [5]:
def load_geovec(path):
    instance = Glove()
    with h5py.File(path, 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)
    instance.word_vectors = v
    instance.no_components = v.shape[1]
    instance.word_biases = np.zeros(v.shape[0])
    instance.add_dictionary(dct)
    return instance

In [6]:
# Stopwords
extra_stopwords = [
    'also',
]
stop = stopwords.words('english') + extra_stopwords

In [7]:
def tokenize(text, min_len=1):
    '''Function that tokenize a set of strings
    Input:
        -text: set of strings
        -min_len: tokens length
    Output:
        -list containing set of tokens'''

    tokens = [word.lower() for sent in nltk.sent_tokenize(text)
              for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if token.isalpha() and len(token) >= min_len:
            filtered_tokens.append(token)

    return [x.lower() for x in filtered_tokens if x not in stop]


def tokenize_and_lemma(text, min_len=0):
    '''Function that retrieves lemmatised tokens
    Inputs:
        -text: set of strings
        -min_len: length of text
    Outputs:
        -list containing lemmatised tokens'''
    filtered_tokens = tokenize(text, min_len=min_len)

    lemmas = [lemma.lemmatize(t) for t in filtered_tokens]
    return lemmas


def get_vector(word, model, return_zero=False):
    '''Function that retrieves word embeddings (vector)
    Inputs:
        -word: token (string)
        -model: trained MLP model
        -return_zero: boolean variable
    Outputs:
        -wv: numpy array (vector)'''
    epsilon = 1.e-10

    unk_idx = model.dictionary['unk']
    idx = model.dictionary.get(word, unk_idx)
    wv = model.word_vectors[idx].copy()

    if return_zero and word not in model.dictionary:
        n_comp = model.word_vectors.shape[1]
        wv = np.zeros(n_comp) + epsilon

    return wv

In [8]:
def mean_embeddings(dataframe_file, model):
    '''Function to retrieve sentence embeddings from dataframe with
    lithological descriptions.
    Inputs:
        -dataframe_file: pandas dataframe containing lithological descriptions
                         and reclassified lithologies
        -model: word embeddings model generated using GloVe
    Outputs:
        -DF: pandas dataframe including sentence embeddings'''
    DF = pd.read_pickle(dataframe_file)
    DF = DF.drop_duplicates(subset=['x', 'y', 'z'])
    DF['tokens'] = DF['Description'].apply(lambda x: tokenize_and_lemma(x))
    DF['length'] = DF['tokens'].apply(lambda x: len(x))
    DF = DF.loc[DF['length']> 0]
    DF['vectors'] = DF['tokens'].apply(lambda x: np.asarray([get_vector(n, model) for n in x]))
    DF['mean'] = DF['vectors'].apply(lambda x: np.mean(x[~np.all(x == 1.e-10, axis=1)], axis=0))
    DF['reclass'] = pd.Categorical(DF.reclass)
    DF['code'] = DF.reclass.cat.codes
    DF['drop'] = DF['mean'].apply(lambda x: (~np.isnan(x).any()))
    DF = DF[DF['drop']]
    return DF

In [9]:
# loading word embeddings model
# (This can be obtained from https://github.com/spadarian/GeoVec )
#modelEmb = Glove.load('/home/ignacio/Documents/chapter2/best_glove_300_317413_w10_lemma.pkl')
modelEmb = load_geovec('geovec_300d_v1.h5')

# getting the mean embeddings of descriptions
DF = mean_embeddings('manualTest_ygsb.pkl', modelEmb)

In [10]:
DF2 = DF[DF['code'].isin(DF['code'].value_counts()[DF['code'].value_counts()>2].index)]
print(DF2)

                      Description  HydroCode  FromDepth  ToDepth     TopElev  \
0            Transported  hardpan     124897          0      1.0  400.000000   
1            Transported  hardpan     124897          1      2.0  399.133975   
2                  Regolith  Clay     124897          2      3.0  398.267949   
3                  Regolith  Clay     124897          3      4.0  397.401924   
4                  Regolith  Clay     124897          4      5.0  396.535898   
...                           ...        ...        ...      ...         ...   
159942                     Basalt    4167632         25     28.0  375.000000   
159943                     Basalt    4167632         28     29.0  372.000000   
159944  Banded Iron Formation BIF    4167633          1      4.0  399.000000   
159945                     Basalt    4167633          4      5.0  396.000000   
159946  Banded Iron Formation BIF    4167634         46     47.0  354.000000   

        BottomElev MajorLithCode  geome

In [11]:
def split_stratified_dataset(Dataframe, test_size, validation_size):
    '''Function that split dataset into test, training and validation subsets
    Inputs:
        -Dataframe: pandas dataframe with sentence mean_embeddings
        -test_size: decimal number to generate the test subset
        -validation_size: decimal number to generate the validation subset
    Outputs:
        -X: numpy array with embeddings
        -Y: numpy array with lithological classes
        -X_test: numpy array with embeddings for test subset
        -Y_test: numpy array with lithological classes for test subset
        -Xt: numpy array with embeddings for training subset
        -yt: numpy array with lithological classes for training subset
        -Xv: numpy array with embeddings for validation subset
        -yv: numpy array with lithological classes for validation subset
        '''
    #df2 = Dataframe[Dataframe['code'].isin(Dataframe['code'].value_counts()[Dataframe['code'].value_counts()>2].index)]
    #X = np.vstack(df2['mean'].values)
    #Y = df2.code.values.reshape(len(df2.code), 1)
    X = np.vstack(Dataframe['mean'].values)
    Y = Dataframe.code.values.reshape(len(Dataframe.code), 1)
    #print(X.shape)
    #print (Dataframe.code.values.shape)
    #print (len(Dataframe.code))
    #print (Y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        Y, stratify=Y,
                                                        test_size=test_size,
                                                        random_state=42)
    #print(X_train.shape)
    #print(Y_train.shape)
    Xt, Xv, yt, yv = train_test_split(X_train,
                                      y_train,
                                      test_size=validation_size,
                                      stratify=None,
                                      random_state=1)
    return X, Y, X_test, y_test, Xt, yt, Xv, yv

In [12]:
# subseting dataset for training classifier
X, Y, X_test, Y_test, X_train, Y_train, X_validation, Y_validation = split_stratified_dataset(DF2, 0.1, 0.1)

# encoding lithological classes
encodes = one_enc.fit_transform(Y_train).toarray()

# MLP model generation
model = Sequential()
model.add(Dense(100, input_dim=300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(units=len(DF2.code.unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# training MLP model
model.fit(X_train, encodes, epochs=30, batch_size=100, verbose=2)

# saving MLP model
model.save('mlp_prob_model.h5')

Epoch 1/30
 - 8s - loss: 0.2053 - accuracy: 0.9599
Epoch 2/30
 - 7s - loss: 0.0070 - accuracy: 0.9988
Epoch 3/30
 - 6s - loss: 0.0036 - accuracy: 0.9994
Epoch 4/30
 - 6s - loss: 0.0034 - accuracy: 0.9995
Epoch 5/30
 - 6s - loss: 0.0033 - accuracy: 0.9993
Epoch 6/30
 - 6s - loss: 0.0024 - accuracy: 0.9996
Epoch 7/30
 - 6s - loss: 0.0020 - accuracy: 0.9997
Epoch 8/30
 - 7s - loss: 0.0016 - accuracy: 0.9998
Epoch 9/30
 - 7s - loss: 0.0042 - accuracy: 0.9992
Epoch 10/30
 - 6s - loss: 0.0023 - accuracy: 0.9996
Epoch 11/30
 - 6s - loss: 0.0017 - accuracy: 0.9997
Epoch 12/30
 - 6s - loss: 0.0016 - accuracy: 0.9998
Epoch 13/30
 - 6s - loss: 0.0033 - accuracy: 0.9993
Epoch 14/30
 - 6s - loss: 0.0019 - accuracy: 0.9996
Epoch 15/30
 - 6s - loss: 0.0015 - accuracy: 0.9998
Epoch 16/30
 - 7s - loss: 0.0015 - accuracy: 0.9998
Epoch 17/30
 - 7s - loss: 0.0016 - accuracy: 0.9998
Epoch 18/30
 - 7s - loss: 0.0016 - accuracy: 0.9998
Epoch 19/30
 - 6s - loss: 0.0016 - accuracy: 0.9998
Epoch 20/30
 - 8s - l

In [13]:
def retrieve_predictions(classifier, x):
    '''Function that retrieves lithological classes using the trained classifier
    Inputs:
        -classifier: trained MLP classifier
        -x: numpy array containing embbedings
    Outputs:
        -codes_pred: numpy array containing lithological classes predicted'''
    preds = classifier.predict(x, verbose=0)
    new_onehot = np.zeros((x.shape[0], 72))
    new_onehot[np.arange(len(preds)), preds.argmax(axis=1)] = 1
    codes_pred = one_enc.inverse_transform(new_onehot)
    return codes_pred


def classifier_assess(classifier, x, y):
    '''Function that prints the performance of the classifier
    Inputs:
        -classifier: trained MLP classifier
        -x: numpy array with embeddings
        -y: numpy array with lithological classes predicted'''
    Y2 = retrieve_predictions(classifier, x)
    print('f1 score: ', metrics.f1_score(y, Y2, average='macro'),
          'accuracy: ', metrics.accuracy_score(y, Y2),
          'balanced_accuracy:', metrics.balanced_accuracy_score(y, Y2))


def save_predictions(Dataframe, classifier, x, name):
    '''Function that saves dataframe predictions as a pickle file
    Inputs:
        -Dataframe: pandas dataframe with mean_embeddings
        -classifier: trained MLP model,
        -x: numpy array with embeddings,
        -name: string name to save dataframe
    Outputs:
        -save dataframe'''
    preds = classifier.predict(x, verbose=0)
    Dataframe['predicted_probabilities'] = preds.tolist()
    Dataframe['pred'] = retrieve_predictions(classifier, x).astype(np.int32)
    Dataframe[['x', 'y', 'FromDepth', 'ToDepth', 'TopElev', 'BottomElev',
               'mean', 'predicted_probabilities', 'pred', 'reclass', 'code']].to_pickle('{}.pkl'.format(name))

In [14]:
# assessment of model performance
classifier_assess(model, X_validation, Y_validation)

# save lithological prediction likelihoods dataframe
save_predictions(DF2, model, X, 'YGSBpredictions')

f1 score:  1.0 accuracy:  1.0 balanced_accuracy: 1.0


In [15]:
import pickle

with open('YGSBpredictions.pkl', 'rb') as f:
    data = pickle.load(f)

In [16]:
print(data)

                  x            y  FromDepth  ToDepth     TopElev  BottomElev  \
0       513822.0449  6952747.766          0      1.0  400.000000  399.133975   
1       513822.5449  6952747.766          1      2.0  399.133975  398.267949   
2       513823.0449  6952747.766          2      3.0  398.267949  397.401924   
3       513823.5449  6952747.766          3      4.0  397.401924  396.535898   
4       513824.0449  6952747.766          4      5.0  396.535898  395.669873   
...             ...          ...        ...      ...         ...         ...   
159942  576956.0811  6882354.387         25     28.0  375.000000  372.000000   
159943  576956.0811  6882354.387         28     29.0  372.000000  371.000000   
159944  577054.2428  6882353.752          1      4.0  399.000000  396.000000   
159945  577054.2428  6882353.752          4      5.0  396.000000  395.000000   
159946  577152.4045  6882353.117         46     47.0  354.000000  353.000000   

                                       

In [17]:
len(data)

122502

In [18]:
data.head()

,x,y,FromDepth,ToDepth,TopElev,BottomElev,mean,predicted_probabilities,pred,reclass,code
0,513822.0449,6952747.766,0,1.0,400.000000,399.133975,"[-0.10188103, -0.009773923, -0.030616358, -0.1...","[1.2841216978642933e-10, 1.470962351923788e-09...",14,colluvium,14
1,513822.5449,6952747.766,1,2.0,399.133975,398.267949,"[-0.10188103, -0.009773923, -0.030616358, -0.1...","[1.2841216978642933e-10, 1.470962351923788e-09...",14,colluvium,14
2,513823.0449,6952747.766,2,3.0,398.267949,397.401924,"[0.09962925, -0.10881818, -0.29363656, -0.2486...","[9.695031105406576e-24, 0.0, 0.0, 1.1913997890...",44,mud,44
3,513823.5449,6952747.766,3,4.0,397.401924,396.535898,"[0.09962925, -0.10881818, -0.29363656, -0.2486...","[9.695031105406576e-24, 0.0, 0.0, 1.1913997890...",44,mud,44
4,513824.0449,6952747.766,4,5.0,396.535898,395.669873,"[0.09962925, -0.10881818, -0.29363656, -0.2486...","[9.695031105406576e-24, 0.0, 0.0, 1.1913997890...",44,mud,44


In [19]:
tmp = data['predicted_probabilities'][0]

In [20]:
len(tmp)

72

In [21]:
#data.to_csv('YGSBpredictions.csv')

In [22]:
import striplog
striplog.__version__

'0.8.4'

In [ ]:
from striplog import Lexicon, Component, Position, Interval, Decor, Legend, Striplog
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
legend = Legend.builtin('NSDOE')
lexicon = Lexicon.default()

In [ ]:
s = Striplog.from_csv(text=data, stop=650)